In [1]:
import os
from os import listdir
from os.path import isfile, join, splitext
from time import sleep
import math
import sys
import requests
import json
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages')
import pandas as pd
import geopandas
import plotly.express as px
import numpy as np
import datetime as dt
import swifter
import geopy
from geopy.distance import geodesic
from geopy.geocoders import OpenMapQuest
from datetime import timedelta
from datetime import datetime

count = 0
bad_count = 0
good_count = 0

tick_employees = [
    'Francisco Nieves',
    'Jose Rosado',
    'Brendon Deveaux',
    'Eduardo Cortez',
    'Seamus O\'connell',
    'Ryan Fratus',
    'Jamir Alexander',
    'Ryan Fleet',
    'John Moudarri',
    'Kevin Enriques-Uluan',
    'Daniel Cook',
    'Marcus Harrington',
    'Luis Alberto Pena',
    'Christopher Polack'
]

cape_guys = [
    'Daniel Cook',
    'Marcus Harrington',
    'Christopher Polack'
]

In [2]:
#geolocator = Nominatim(user_agent='pure-solutions')

def get_multiple_csv_from_folder(folder_path: str):
    onlyfiles = [f for f in listdir(folder_path) if (isfile(join(folder_path, f)) and splitext(f)[1]=='.csv')]
    return onlyfiles

def import_csv(append_df: pd.DataFrame, folder_path: str, file_name: str):
    df = pd.DataFrame(pd.read_csv(f'{folder_path}/{file_name}'))
    df['Date'] = datetime.strptime(f"{file_name.split('_')[1].split('.')[0]}/2021", '%m%d/%Y')
    df = df.loc[~(df['StartTime']==0) & ~(df['EndTime']==0)]
    df['StartTimeFormatted'] = df.apply(lambda row: datetime.strptime(str(row['StartTime']), '%H%M'), axis=1)
    df['EndTimeFormatted'] = df.apply(lambda row: datetime.strptime(str(row['EndTime']), '%H%M'), axis=1)
    return df[['EmployeeName', 'Date', 'StartTimeFormatted', 'EndTimeFormatted', 'Address']]

def clean_time(row):
    row_list = row['InOutTimeFormatted'].split(' ')
    row_list.remove('/')
    try:
        row_list = [row_list[0]+row_list[1].upper(), row_list[2]+row_list[3].upper()]
    except:
        row_list = None
    return row_list

def get_time(row, start: bool):
    row_list = row['InOutTimeFormatted'].split(' ')
    row_list.remove('/')
    try:
        row_list = [row_list[0]+row_list[1].upper(), row_list[2]+row_list[3].upper()]
        if start:
            time = datetime.strptime(row_list[0], '%I:%M%p')
        else:
            time = datetime.strptime(row_list[1], '%I:%M%p')
        return time
    except:
        return None

def geocode(row, address):
    location = geolocator.geocode(address)
    if location is None:
        pass
    else:
        try:
            if row.name[0] in cape_guys:
                dist = geodesic((location.latitude, location.longitude), (41.6955182350886, -69.96357346008841)).miles
            else:
                dist = geodesic((location.latitude, location.longitude), (42.36721418348649, -71.31387091988164)).miles
            print(dist)
            return dist
        except:        
            return None

In [3]:
geolocator = OpenMapQuest(api_key='qdcSGA0YMU9EUWfrfFtQ08jAGNPRB63c')
def geocode_twoAddresses(df, row, address1, address2):
    global count
    global bad_count
    count += 1
    dist = None
    routes = None
    print('------------------------------')
    location_start = geolocator.geocode(address1)
    location_end = geolocator.geocode(address2)
    try:
        if (address1 is None) or (address2 is None):
            print(f'''Start or End Location is None ::: {location_start} ::: {location_end}''')
            #bad_count += 1
            dist = None
        else:
            try:
                r = requests.get(
                    f"http://router.project-osrm.org/route/v1/car/{location_start.longitude},{location_start.latitude};{location_end.longitude},{location_end.latitude}?overview=false"""
                )
                routes = json.loads(r.content)
                #dist = geodesic((location_start.latitude, location_start.longitude), (location_end.latitude, location_end.longitude)).miles
                dist = routes.get('routes')[0]['distance']*0.000621371
                #print(f'''{dist:,.2f} ::: {count}/{len(df)}''')
                if dist >= 100:
                    dist = None
                    print(f'BIG WEIRD NUMBER. MAKE NO SENSE ::: {location_start} ::: {location_end}\n{type(address1)} ::: {type(address2)}')
                    #bad_count += 1
            except Exception as e:
                print(f'''Can\'t get routes ::: {address1} ::: {address2} :::\n{location_start} ::: {location_end}''')
                print(str(e))
                #bad_count += 1
    except:
        print(f'''Couldn't find Address ::: {address1} ::: {address2}''')
    print(f'Returned: {dist}')
    print(f'''Entries Failed: {(bad_count/count)*100:,.2f}%''')
    print('------------------------------')
    return dist

def time_to_next_service(df, row):
    print('------------------------------')
    try:
        leave_time = list(df['EndTimeFormatted'].loc[(df['NextAddress']==row['Address']) & (df['Date']==row['Date'])])[0]
        arrive_time = row['StartTimeFormatted']
        print(f'''{leave_time} ::: {arrive_time} ::: {arrive_time-leave_time}''')
        return float(((arrive_time - leave_time).days*24) + ((arrive_time - leave_time).seconds/3600))
    except IndexError:
        print(f'''FAIL \n{row['Address']} ::: {row['NextAddress']} \n{row['Date']} ::: {row['MilesToNextService']}''')
        return None
    print('------------------------------')

In [4]:
csv = pd.read_csv('/Users/fabian_coll/Desktop/Pure-Solutions/data/dirty/tech_recap/services_with_addresses.csv')
df = pd.DataFrame(csv)
df = df.drop(columns=['Unnamed: 0'])
df = df.loc[df['EmployeeName'].isin(tick_employees)]
df = df.dropna()
print(df.columns)
print(df.dtypes)

Index(['EmployeeName', 'StartTime', 'EndTime', 'Date', 'Address',
       'NextAddress'],
      dtype='object')
EmployeeName    object
StartTime       object
EndTime         object
Date            object
Address         object
NextAddress     object
dtype: object


In [ ]:
df['MilesToNextService'] = df.apply(lambda row: geocode_twoAddresses(df, row, row['Address'], row['NextAddress']), axis=1)

In [25]:
df['StartTimeFormatted'] = pd.to_datetime(df['StartTimeFormatted'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%H:%M')
df['StartTimeFormatted'] = pd.to_datetime(df['StartTimeFormatted'], format='%H:%M')
df['EndTimeFormatted'] = pd.to_datetime(df['EndTimeFormatted'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%H:%M')
df['EndTimeFormatted'] = pd.to_datetime(df['EndTimeFormatted'], format='%H:%M')
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
[print(f'{name} ::: {dtype}') for name, dtype in df.dtypes.iteritems()]

EmployeeName ::: object
StartTimeFormatted ::: datetime64[ns]
EndTimeFormatted ::: datetime64[ns]
Date ::: datetime64[ns]
Address ::: object
NextAddress ::: object
MilesToNextService ::: float64
TimeToNextService ::: float64
SpeedToNextService ::: float64


[None, None, None, None, None, None, None, None, None]

In [26]:
df['TimeToNextService'] = df.apply(lambda row: time_to_next_service(df, row), axis=1)

------------------------------
FAIL 
5 Wayside Rd, Wayland, MA 01778 ::: 2 Winthrop Ter, Wayland, MA 01778-0000 
2021-07-20 00:00:00 ::: 2.3538776222
------------------------------
1900-01-01 12:34:00 ::: 1900-01-01 13:04:00 ::: 0 days 00:30:00
------------------------------
FAIL 
10 Sundance Way, Natick, MA 01760 ::: 15 Drury Ln, Natick, MA 01760 
2021-07-20 00:00:00 ::: 4.0181577086
------------------------------
1900-01-01 15:08:00 ::: 1900-01-01 15:16:00 ::: 0 days 00:08:00
------------------------------
FAIL 
126 Clark Ave, Wellfleet, MA 02667 ::: 130 Marven Way, Wellfleet, MA 02667 
2021-07-20 00:00:00 ::: 1.9209063094
------------------------------
FAIL 
16 Snows Rd, Truro, MA 02666 ::: 2 Higgins Hollow Rd, Truro, MA 02655 
2021-07-20 00:00:00 ::: 0.8300895189
------------------------------
1900-01-01 11:38:00 ::: 1900-01-01 11:49:00 ::: 0 days 00:11:00
------------------------------
1900-01-01 12:05:00 ::: 1900-01-01 12:17:00 ::: 0 days 00:12:00
------------------------------
F

In [28]:
df['SpeedToNextService'] = df['MilesToNextService'] / df['TimeToNextService']
df = df.replace([np.inf, -np.inf], None)
df = df.loc[(df['SpeedToNextService'] != 0) & (df['SpeedToNextService'] <= 65)]
df = df.dropna()
#df = df.groupby(by=['EmployeeName']).mean()
print(df)
df.to_csv('/Users/fabian_coll/Desktop/Pure-Solutions/data/dirty/tech_recap/inbetween_services.csv')

              EmployeeName  StartTimeFormatted    EndTimeFormatted       Date  \
1          Brendon Deveaux 1900-01-01 13:04:00 1900-01-01 13:17:00 2021-07-20   
3              Daniel Cook 1900-01-01 15:16:00 1900-01-01 15:40:00 2021-07-20   
6           Eduardo Cortez 1900-01-01 11:49:00 1900-01-01 12:05:00 2021-07-20   
7           Eduardo Cortez 1900-01-01 12:17:00 1900-01-01 12:32:00 2021-07-20   
9           Eduardo Cortez 1900-01-01 11:21:00 1900-01-01 11:38:00 2021-07-20   
...                    ...                 ...                 ...        ...   
1587         John Moudarri 1900-01-01 14:18:00 1900-01-01 14:28:00 2021-07-06   
1588           Jose Rosado 1900-01-01 13:02:00 1900-01-01 13:12:00 2021-07-06   
1590           Jose Rosado 1900-01-01 12:43:00 1900-01-01 12:57:00 2021-07-06   
1591  Kevin Enriques-Uluan 1900-01-01 10:16:00 1900-01-01 10:35:00 2021-07-06   
1592  Kevin Enriques-Uluan 1900-01-01 15:38:00 1900-01-01 16:07:00 2021-07-06   

                           